In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from tensorflow import keras
from sklearn.model_selection import train_test_split
import cv2

In [2]:
data_dir = r"D:\Datasets\gtsrb\gtsrb"
num_categories = len(os.listdir(data_dir))
def load_data(data_dir):
    images = []
    labels = []
    for i in range(num_categories):
        di = os.path.join(data_dir, f"{str(i)}")
        for img in os.listdir(di):
            path = os.path.join(data_dir,f"{str(i)}", img)
            im = cv2.imread(path)
            im_res = cv2.resize(im, (32, 32), interpolation= cv2.INTER_AREA)
            labels.append(i)
            images.append(im_res)
    return (images, labels,)

images, labels = load_data(data_dir)
labels = tf.keras.utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(
np.array(images, dtype=np.float32), np.array(labels), test_size=0.2
)

In [3]:
class Resnet(keras.Model):
    def __init__(self, num_classes):
        super(Resnet, self).__init__()
        self.conv1 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3))
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(128, (3, 3), activation='relu') 
        self.bn2 = layers.BatchNormalization()
        
        self.res_conv1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')
        self.res_conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')
        
        self.conv3 = layers.Conv2D(256, (3, 3), activation='relu')
        self.bn3 = layers.BatchNormalization()
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(256, activation='relu')
        self.fc2 = layers.Dense(num_classes, activation='softmax')
    
    def conv_block1(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x) 
        return x
    
    def res_block1(self, r):
        x = self.res_conv1(r)
        x = self.res_conv2(x) + r
        return x
        
    def conv_block2(self, x):
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
    def call(self, x):
        x = self.conv_block1(x)
        x = self.res_block1(x)
        x = self.conv_block2(x)
        return x

In [4]:
net = Resnet(num_categories)
net.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.categorical_accuracy])
net.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=3)

Epoch 1/3
666/666 [==============================] - 59s 88ms/step - loss: 1.2690 - categorical_accuracy: 0.8518 - val_loss: 0.2492 - val_categorical_accuracy: 0.9384
Epoch 2/3
666/666 [==============================] - 57s 86ms/step - loss: 0.1075 - categorical_accuracy: 0.9732 - val_loss: 0.4775 - val_categorical_accuracy: 0.8833
Epoch 3/3
666/666 [==============================] - 56s 85ms/step - loss: 0.1167 - categorical_accuracy: 0.9718 - val_loss: 0.4278 - val_categorical_accuracy: 0.9154
